## **Librerias**

In [39]:
import pandas as pd
import numpy as np
import plotly.express as px

## **Preparación de Datos**

In [40]:
df_closed = pd.read_csv("../data/raw/olist_closed_deals_dataset.csv")
#
df_mkt = pd.read_csv("../data/raw/olist_marketing_qualified_leads_dataset.csv")

In [3]:
df_mkt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mql_id              8000 non-null   object
 1   first_contact_date  8000 non-null   object
 2   landing_page_id     8000 non-null   object
 3   origin              7940 non-null   object
dtypes: object(4)
memory usage: 250.1+ KB


In [17]:
df_closed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   mql_id                         842 non-null    object 
 1   seller_id                      842 non-null    object 
 2   sdr_id                         842 non-null    object 
 3   sr_id                          842 non-null    object 
 4   won_date                       842 non-null    object 
 5   business_segment               841 non-null    object 
 6   lead_type                      836 non-null    object 
 7   lead_behaviour_profile         665 non-null    object 
 8   has_company                    63 non-null     object 
 9   has_gtin                       64 non-null     object 
 10  average_stock                  66 non-null     object 
 11  business_type                  832 non-null    object 
 12  declared_product_catalog_size  69 non-null     flo

In [10]:
df_mkt['mql_id'].nunique()

8000

In [18]:
def get_low_completion_columns(df, seller_id_col='seller_id', threshold=80):
    """
    Retorna las columnas que tienen un porcentaje de completitud menor al threshold especificado,
    calculado en relación a la cantidad de seller_id únicos.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame a analizar
    seller_id_col : str
        Nombre de la columna que contiene el seller_id
    threshold : int
        Porcentaje mínimo de completitud requerido (0-100)
    
    Returns:
    --------
    list
        Lista de columnas que no cumplen con el porcentaje mínimo de completitud
    """
    
    # Cantidad de sellers únicos
    total_sellers = df[seller_id_col].nunique()
    
    # Calcular porcentaje de completitud para cada columna
    completion_rates = (df.count() / total_sellers * 100).round(2)
    
    # Filtrar columnas por debajo del threshold
    low_completion_cols = completion_rates[completion_rates < threshold].index.tolist()
    
    return low_completion_cols

## **Limipieza, Formato y Features**

In [24]:
df_mkt_closed = df_mkt.merge(df_closed, on='mql_id', how='left')
#
# Usar el análisis
# Asumiendo que tu DataFrame se llama 'df'
low_quality_columns = get_low_completion_columns(df_mkt_closed, threshold=60)
#
list_drop_columns = ['landing_page_id', 'seller_id', 'sdr_id'] + low_quality_columns
#
df_mkt_closed = df_mkt_closed.drop(columns=list_drop_columns)
#
df_processed = df_mkt_closed.assign(
        first_contact_date=lambda df: pd.to_datetime(df_mkt_closed['first_contact_date'], format='%Y-%m-%d', errors='coerce'),
        won_date= pd.to_datetime(df_mkt_closed['won_date'].str[:10], format='%Y-%m-%d', errors='coerce'),
        target= lambda df: np.where(df_mkt_closed['won_date'].isnull(), 0, 1),
        origin= lambda df: np.where(df_mkt_closed['origin'].isnull(), 'unknown', df_mkt_closed['origin']))
#
df_processed['days_to_convert'] = (df_processed['won_date'] - df_processed['first_contact_date']).dt.days

### **Funnel**

In [25]:
# Total de MQLs (todos los leads en el dataset)
n_mql = df_mkt_closed['mql_id'].nunique()

# SQLs: los que tienen un vendedor asignado (ej: sr_id no nulo)
n_sql = df_mkt_closed[df_mkt_closed['sr_id'].notna()]['mql_id'].nunique()

# Won: los que tienen una fecha de conversión
n_won = df_mkt_closed[df_mkt_closed['won_date'].notna()]['mql_id'].nunique()

print("MQL:", n_mql)
print("SQL:", n_sql)
print("Won:", n_won)

MQL: 8000
SQL: 842
Won: 842


In [26]:
conversion_mql_to_sql = n_sql / n_mql
conversion_sql_to_won = n_won / n_sql
conversion_mql_to_won = n_won / n_mql

print(f"Tasa MQL → SQL: {conversion_mql_to_sql:.2%}")
print(f"Tasa SQL → Won: {conversion_sql_to_won:.2%}")
print(f"Tasa MQL → Won: {conversion_mql_to_won:.2%}")

Tasa MQL → SQL: 10.53%
Tasa SQL → Won: 100.00%
Tasa MQL → Won: 10.53%


## **Análisis de Conversión y Atribución por Origen**

In [52]:
origin_conversion = df_processed.query("origin != 'unknown'")\
                              .groupby('origin', as_index=False)\
                              .agg(mql=('mql_id', 'count'), won=('target', 'sum'),
                                   days_to_convert_q3=('days_to_convert', lambda x: x.quantile(0.75)))
#
# Calcular los porcentajes
total_mql = origin_conversion['mql'].sum()
total_won = origin_conversion['won'].sum()
# Añadir columnas de porcentaje
origin_conversion['mql_percentage'] = (origin_conversion['mql'] / total_mql * 100).round(2)
origin_conversion['won_percentage'] = (origin_conversion['won'] / total_won * 100).round(2)
origin_conversion['conversion'] = (origin_conversion['won'] / origin_conversion['mql']  * 100).round(2)
origin_conversion['weighted_conversion'] = (origin_conversion['won'] / origin_conversion['mql']) * np.log(origin_conversion['mql'])
#
origin_conversion = origin_conversion.sort_values(by='conversion', ascending=False)

In [53]:
origin_conversion

,origin,mql,won,days_to_convert_q3,mql_percentage,won_percentage,conversion,weighted_conversion
6,paid_search,1586,195,80.00,23.18,30.05,12.30,0.906021
3,organic_search,2296,271,55.50,33.56,41.76,11.80,0.913436
0,direct_traffic,499,56,32.25,7.29,8.63,11.22,0.697206
7,referral,284,24,35.25,4.15,3.70,8.45,0.477378
8,social,1350,75,84.00,19.73,11.56,5.56,0.400437
1,display,118,6,14.50,1.72,0.92,5.08,0.242577
5,other_publicities,65,3,41.50,0.95,0.46,4.62,0.192664
2,email,493,15,62.00,7.21,2.31,3.04,0.188656
4,other,150,4,20.25,2.19,0.62,2.67,0.133617


In [54]:
px.box(df_processed.query("origin != 'unknown'"), x='origin', y='days_to_convert', color='origin')

## **Creación de Metodología Score Importance**
- weighted_conversion y days_to_convert_q3

In [46]:
def calculate_origin_score(df, weight_conversion=0.6, weight_speed=0.4):
    """
    Calculates a combined score for origin performance based on
    weighted conversion efficiency and conversion speed (days_to_convert_q3).

    Args:
        df (pd.DataFrame): DataFrame containing 'origin', 'weighted_conversion',
                           and 'days_to_convert_q3'.
        weight_conversion (float): Weight for the weighted_conversion component (0 to 1).
        weight_speed (float): Weight for the days_to_convert_q3 component (0 to 1).
                                Should sum to 1 with weight_conversion.

    Returns:
        pd.DataFrame: Original DataFrame with added normalized columns and the final 'origin_score'.
                      Sorted by 'origin_score' descending.
    """
    if not np.isclose(weight_conversion + weight_speed, 1.0):
        raise ValueError("Weights must sum to 1.0")

    # --- 1. Normalization (Manual Min-Max Scaling 0-1) ---
    # Create copies to avoid SettingWithCopyWarning if df is a slice
    df_scored = df.copy()

    # a) Weighted Conversion (Higher is better)
    min_wc = df_scored['weighted_conversion'].min()
    max_wc = df_scored['weighted_conversion'].max()
    if max_wc > min_wc:
        df_scored['norm_weighted_conversion'] = (df_scored['weighted_conversion'] - min_wc) / (max_wc - min_wc)
    else:
        df_scored['norm_weighted_conversion'] = 0.5 # Assign mid-value if all are the same

    # b) Days to Convert Q3 (Lower is better - INVERTED scale)
    min_days = df_scored['days_to_convert_q3'].min()
    max_days = df_scored['days_to_convert_q3'].max()
    if max_days > min_days:
        # Inverted scale: (max - value) / (max - min)
        df_scored['norm_days_to_convert'] = (max_days - df_scored['days_to_convert_q3']) / (max_days - min_days)
    else:
        df_scored['norm_days_to_convert'] = 0.5 # Assign mid-value if all are the same

    # Handle potential NaNs introduced if min/max were NaN or during calculation
    df_scored['norm_weighted_conversion'] = df_scored['norm_weighted_conversion'].fillna(0)
    df_scored['norm_days_to_convert'] = df_scored['norm_days_to_convert'].fillna(0)


    # --- 2. Weighted Combination ---
    df_scored['origin_score'] = (weight_conversion * df_scored['norm_weighted_conversion'] +
                                 weight_speed * df_scored['norm_days_to_convert'])
    #
    list_cols_sel = ['origin', 'mql', 'conversion', 'weighted_conversion', 'norm_weighted_conversion', 'norm_days_to_convert', 'origin_score']
    # --- 3. Return sorted DataFrame ---
    return df_scored.sort_values(by='origin_score', ascending=False)[list_cols_sel]

In [47]:
scored_origins = calculate_origin_score(origin_conversion, weight_conversion=0.6, weight_speed=0.4)

In [48]:
scored_origins

,origin,mql,conversion,weighted_conversion,norm_weighted_conversion,norm_days_to_convert,origin_score
3,organic_search,2296,11.80,0.913436,1.000000,0.410072,0.764029
0,direct_traffic,499,11.22,0.697206,0.722718,0.744604,0.731473
6,paid_search,1586,12.30,0.906021,0.990492,0.057554,0.617317
7,referral,284,8.45,0.477378,0.440822,0.701439,0.545069
1,display,118,5.08,0.242577,0.139725,1.000000,0.483835
4,other,150,2.67,0.133617,0.000000,0.917266,0.366906
5,other_publicities,65,4.62,0.192664,0.075719,0.611511,0.290036
8,social,1350,5.56,0.400437,0.342156,0.000000,0.205294
2,email,493,3.04,0.188656,0.070580,0.316547,0.168967


In [55]:
# Crear gráfico de burbujas
fig = px.scatter(
    scored_origins,
    x="norm_weighted_conversion",           # Eje X
    y="norm_days_to_convert",               # Eje Y
    size="mql",                             # Tamaño de burbuja según cantidad de MQLs
    color="origin_score",                   # Color por canal
    hover_name="origin",                    # Mostrar nombre al pasar el mouse
    size_max=60,                            # Tamaño máximo de burbuja
    title="Comparación de Orígenes de MQLs",
)

# Agregar línea horizontal punteada en y=0.5
fig.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.7)

# Agregar línea vertical punteada en x=0.5
fig.add_vline(x=0.5, line_dash="dash", line_color="gray", opacity=0.7)

fig.show()